In [1]:
import sys
sys.path.append("..")
import argparse
import numpy as np
import dgl
from dgl import DGLGraph
import torch
import torch.nn.functional as F
import time
import matplotlib.pyplot as plt
import pandas as pd
from tqdm import tqdm
from collections import Counter
import pickle
import h5py
import random
import glob2
import seaborn as sns

import train
import models

%load_ext autoreload
%autoreload 2


Using backend: pytorch


In [2]:
device = torch.device("cuda:0" if torch.cuda.is_available() == True else 'cpu')
print(device)

cuda:0


In [3]:
import warnings
warnings.filterwarnings("ignore")

import torch
import numpy as np
import scanpy as sc
from torch.utils.data import DataLoader
from sklearn.model_selection import train_test_split

In [4]:
import math
import torch
import random
import numpy as np
import copy
from typing import Optional
from sklearn import metrics
from torch import nn, Tensor
from torch.nn import functional as F
from torch.nn.modules import MultiheadAttention, Linear, Dropout, BatchNorm1d, TransformerEncoderLayer


def setup_seed(seed):
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    np.random.seed(seed)
    random.seed(seed)
    torch.backends.cudnn.deterministic = True


np.set_printoptions(precision=4)
np.set_printoptions(suppress=True)    

In [5]:
epochs = 200
run_times = 1

batch_size = 128 
pca_size = 100

In [6]:
results = []
model_name = "GraphConv"
normalize_weights = "none"
node_features = "none"
same_edge_values = False
edge_norm = True
hidden_relu = False
hidden_bn = False
n_layers = 1
hidden_dim = 200
hidden = [300]
nb_genes = 3000
activation = F.relu

path='./Baron.h5'
data = sc.read_h5ad(path)

X_all = data.X
y_all = data.obs.values[:,0]


Y = y_all
X = X_all
n_clusters = len(np.unique(Y))


t0 = time.time()
graph = train.make_graph(
    X,
    Y,
    dense_dim=pca_size,
    node_features=node_features,
    normalize_weights=normalize_weights,
)

labels = graph.ndata["label"]
train_ids = np.where(labels != -1)[0]

sampler = dgl.dataloading.MultiLayerFullNeighborSampler(n_layers)

dataloader = dgl.dataloading.NodeDataLoader(
    graph,
    train_ids,
    sampler,
    batch_size=batch_size,
    shuffle=True,
    drop_last=False,
    num_workers=1,
)


In [ ]:
t1 = time.time()

for run in range(run_times):
    setup_seed(run)
    t_start = time.time()
    torch.manual_seed(run)
    torch.cuda.manual_seed_all(run)
    np.random.seed(run)
    random.seed(run)

    model = models.GCNAE(
        in_feats=pca_size,
        n_hidden=hidden_dim,
        n_layers=n_layers,
        activation=activation,
        dropout=0.1,
        hidden=hidden,
        hidden_relu=hidden_relu,
        hidden_bn=hidden_bn,
    ).to(device)
    if run == 0:
        print(f">", model)

    optim = torch.optim.Adam(model.parameters(), lr=1e-5)

    scores = train.train(model,
                         optim,
                         epochs,
                         dataloader,
                         n_clusters,
                         plot=False,
                         cluster=["KMeans"])
    results.append(scores)
#         results.to_pickle(
#             f"../output/pickle_results/{category}/{category}_gae.pkl")
#         print("Done")
